# Import libraries

In [3]:
import os
import k3d
import json
import numpy as np
from tqdm import tqdm
from easydict import EasyDict as edict

# Specifying trained model

In [4]:
EXP_NAME = '1b5fabcd-9b11-4188-b193-1e0c052ba0f4'

with open(f'experiments/{EXP_NAME}/config.json') as json_file:
    config = json.load(json_file)

config = edict(config)

os.makedirs(f'experiments/{EXP_NAME}/visualizations')

# Visualizating

In [6]:
for subject in tqdm(config.subjects):
    
    with open(f'experiments/{EXP_NAME}/predictions/{subject}.json') as json_file:
        net_output = json.load(json_file)
        
    coordinates = np.array(net_output['coordinates'])
    predictions = np.array(net_output['predictions'])
    labels = np.array(net_output['labels'])
    air_maks = np.array(net_output['air_maks'])

    coordinates = coordinates*(1-air_maks[:, None]).astype(np.float32)
    predictions = predictions*(1-air_maks)
    predictions = (predictions > 0.5).astype(np.float32)
    labels = labels*(1-air_maks)
    tp = np.logical_and(labels, predictions)
    
    pred_colors = np.zeros(len(predictions))

    pred_colors = [0xFFFF00 if (labels[i] == 1) else x for i, x in enumerate(pred_colors)]
    pred_colors = [0xFF9900 if (predictions[i] == 1) else x for i, x in enumerate(pred_colors)]
    pred_colors = [0x00FF00 if (tp[i] == 1) else x for i, x in enumerate(pred_colors)]
    pred_colors = [0xCCCCCC if (x == 0) else x for i, x in enumerate(pred_colors)]

    opacities = [0.5 if (x == 0xCCCCCC) else 1 for x in pred_colors]
    point_sizes = [1 if (x == 0x00FF00) else 0.5 for x in pred_colors]
    
    
    plot = k3d.plot(name='points')
    plt_points = k3d.points(positions=coordinates,
                            colors=pred_colors,
                            opacities=opacities,
                            point_sizes=point_sizes)
    plot += plt_points
    plt_points.shader='3d'

    with open(f'experiments/{EXP_NAME}/visualizations/{subject}.html', 'w') as f:
        f.write(plot.get_snapshot(9))

  0%|          | 0/81 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
100%|██████████| 81/81 [03:15<00:00,  2.41s/it]
